# Assignment 2 for Group44

In [19]:
from typing import List
import heapq


def solve(grid: List[List[str]]) -> int:
    """Your solution to the problem goes in this function.
    :param:
        grid (List[List[str]]): The warehouse layout, e.g., [["@", "@", "@"], ["@", "R", "$"], ["@", "@", "T"]]
    :return:
        int: the minimum number of pushes required for the robot to move the item to an empty shelf.
        return -1 if no solution
        return -2 if invalid input
        return the minimum number of pushes required for the robot to move the item to an empty shelf
    """
    # if the input is not 2d array of strings, return -2
    if not isinstance(grid, list) or not all(isinstance(row, list) for row in grid) or not all(
            isinstance(item, str) for row in grid for item in row):
        print('Invalid input: not a 2d array of strings')
        return -2
    # if the input is not a valid grid, contain charactors other than @#$RT return -2
    if not all(item in ['@', '#', '$', 'R', 'T'] for row in grid for item in row):
        print('Invalid input: not a valid grid, contain charactors other than @#$RT')
        return -2
    # if the input is not a valid grid, contain more than one robot return -2
    if sum(row.count('R') for row in grid) != 1:
        print('Invalid input: not a valid grid, contain more than one robot')
        return -2
    # if there are more than on item, return -2
    if sum(row.count('$') for row in grid) != 1:
        print('Invalid input: not a valid grid, contain more than one item')
        return -2
    # if there are no target shelf, return -2
    if sum(row.count('T') for row in grid) == 0:
        print('Invalid input: not a valid grid, contain no target shelf')
        return -2
    
    # Find the robot, item, and shelves
    robot = item = shelves = None
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 'R':
                robot = (i, j)
            elif grid[i][j] == '$':
                item = (i, j)
            elif grid[i][j] == 'T':
                if shelves is None:
                    shelves = []
                shelves.append((i, j))

    # if the x and y of the item is both blocked, return -1
    if block_x(grid, item[0], item[1]) and block_y(grid, item[0], item[1]):
        print('Invalid input: the item is blocked')
        return -1

    # if the item is surrounded by blocks, return -1
    if surrounded_by_blocks(grid, item[0], item[1]):
        print('Invalid input: the item is surrounded by blocks')
        return -1

    # if all the shelves are blocked, return -1
    if all(surrounded_by_blocks(grid, shelf[0], shelf[1]) for shelf in shelves):
        print('Invalid input: all the shelves are blocked')
        return -1

    # if the distance between the item and the target shelf is 1, return 1
    if any(heuristic(item, shelf) == 1 for shelf in shelves):
        return 1

    return find_min_pushes(grid, robot, item, shelves)


def surrounded_by_blocks(grid, x, y) -> bool:
    """
    Check if the item is surrounded by blocks and edge of the grid
    :param grid: 2d strings
    :param x: x coordinate of the item
    :param y: y coordinate of the item
    :return: Boolean
    """
    if ((grid[x - 1][y] == '@' or x - 1 < 0) and (grid[x + 1][y] == '@' or x + 1 >= len(grid))
            and (grid[x][y - 1] == '@' or y - 1 < 0) and (grid[x][y + 1] == '@' or y + 1 >= len(grid[0]))):
        return True
    return False


def block_x(grid, x, y) -> bool:
    """
    Check if the item is blocked in x direction( either by blocks or edge of the grid)
    :param grid: 2d array strings
    :param x: x coordinate of the item
    :param y: y coordinate of the item
    :return: Boolean
    """
    if grid[x - 1][y] == '@' or grid[x + 1][y] == '@' or x + 1 >= len(grid) or x - 1 < 0:
        return True
    return False


def block_y(grid, x, y) -> bool:
    """
    Check if the item is blocked in y direction( either by blocks or edge of the grid)
    :param grid: 2d array strings
    :param x: x coordinate of the item
    :param y: y coordinate of the item
    :return: Boolean
    """
    if grid[x][y - 1] == '@' or grid[x][y + 1] == '@' or y + 1 >= len(grid[0]) or y - 1 < 0:
        return True
    return False


def find_min_pushes(grid, _robot, item, shelves) -> int:
    """
    Find the minimum pushes required for the robot to move the item to an empty shelf
    :param grid: 2d list of strings
    :param _robot: (x,y) coordinate of the robot
    :param item: (x,y) coordinate of the item
    :param shelves: a list of (x,y) coordinates of all shelves
    :return: int, minimal steps required
    """
    min_pushes = float('inf')
    for shelf in shelves:
        _, cost_item_to_shelf = astar(grid, item, shelf)
        pushes = cost_item_to_shelf.get(shelf, float('inf'))
        min_pushes = min(min_pushes, pushes)

    return min_pushes if min_pushes != float('inf') else -1


def heuristic(a, b) -> int:
    """
    Manhattan distance
    :param a: (x,y) coordinate of the item
    :param b: (x,y) coordinate of the target
    :return: int, Manhattan distance
    """
    return abs(b[0] - a[0]) + abs(b[1] - a[1])


def astar(grid, start, goal) -> (dict, dict):
    """
    A* algorithm
    :param grid: 2d list of strings 
    :param start: start point
    :param goal: target point
    :return: came_from: dict, cost_so_far: dict
    """
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}

    while frontier:
        _, current = heapq.heappop(frontier)

        if current == goal:
            break

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            next = (current[0] + dx, current[1] + dy)
            if 0 <= next[0] < len(grid) and 0 <= next[1] < len(grid[0]) and grid[next[0]][next[1]] != '@':
                new_cost = cost_so_far[current] + 1
                if next not in cost_so_far or new_cost < cost_so_far[next]:
                    cost_so_far[next] = new_cost
                    priority = new_cost + heuristic(goal, next)
                    heapq.heappush(frontier, (priority, next))
                    came_from[next] = current

    return came_from, cost_so_far


In [12]:
# test case 1
grid1 = [["@", "@", "@", "@", "@", "@"],
         ["@", "@", "@", "@", "T", "@"],
         ["@", "#", "$", "#", "#", "@"],
         ["@", "#", "@", "@", "#", "@"],
         ["@", "R", "#", "#", "#", "@"],
         ["@", "T", "@", "@", "@", "@"]]
answer1 = 3
result1 = solve(grid1)
assert result1 == answer1, f"Test case 1: expected {answer1}, got {result1}"
print('Passed test case 1...')

1
Passed test case 1...


In [13]:
# test case 2
grid2 = [["@", "T", "@", "@", "@", "@"],
         ["@", "#", "@", "@", "@", "@"],
         ["@", "#", "#", "#", "$", "@"],
         ["@", "#", "@", "@", "@", "@"],
         ["@", "R", "#", "#", "T", "@"],
         ["@", "@", "@", "@", "@", "@"]]
answer2 = -1
result2 = solve(grid2)
assert result2 == answer2, f"Test case 2: expected {answer2}, got {result2}"
print('Passed test case 2...')

1
Invalid input: the item is blocked
Passed test case 2...


In [20]:
# test case 3
grid3 = [["@", "T", "@", "@", "@", "@"],
         ["@", "#", "@", "@", "@", "@"],
         ["@", "#", "#", "#", "$", "@"],
         ["@", "#", "@", "@", "$", "@"],
         ["@", "R", "#", "#", "T", "@"],
         ["@", "@", "@", "@", "@", "@"]]
answer3 = -2
result3 = solve(grid3)
assert result3 == answer3, f"Test case 3: expected {answer3}, got {result3}"
print('Passed test case 3...')

2
Invalid input: not a valid grid, contain more than one item
Passed test case 3...
